In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
from elo import Elo
import time
from glicko import Glicko
from glicko2 import Glicko2

In [21]:
matches = pd.read_csv("Games.csv", usecols=range(0, 10))
matches = matches[matches["gameId"].between(22200000, 22400000)]
matches["hometeamId"] -= 1610612737
matches["awayteamId"] -= 1610612737
matches = matches.iloc[::-1]
id1 = matches["hometeamId"].to_numpy()
id2 = matches["awayteamId"].to_numpy()
outcomes = matches["homeScore"].to_numpy() > matches["awayScore"].to_numpy()
names = np.array([
    "Atlanta Hawks", "Boston Celtics", "Cleveland Cavaliers", "New Orleans Pelicans", "Chicago Bulls", "Dallas Mavericks", "Denver Nuggets", 
    "Golden State Warriors", "Houston Rockets", "Los Angeles Clippers", "Los Angeles Lakers", "Miami Heat", "Milwaukee Bucks", 
    "Minnesota Timberwolves", "Brooklyn Nets", "New York Knicks", "Orlando Magic", "Indiana Pacers", "Philadelphia 76ers", 
    "Phoenix Suns", "Portland Trail Blazers", "Sacramento Kings", "San Antonio Spurs", "Oklahoma City Thunder", "Toronto Raptors", 
    "Utah Jazz", "Memphis Grizzlies", "Washington Wizards", "Detroit Pistons", "Charlotte Hornets"
])

In [22]:
print(matches)
# target = "Atlanta"
# total_count = matches["hometeamCity"].str.count(rf'\b{target}\b').sum()
# total_count += matches["awayteamCity"].str.count(rf'\b{target}\b').sum()
# print(total_count)

        gameId             gameDate   hometeamCity  hometeamName  hometeamId  \
4087  22200001  2022-10-18 19:30:00         Boston       Celtics           1   
4086  22200002  2022-10-18 22:00:00   Golden State      Warriors           7   
4085  22200004  2022-10-19 19:00:00        Indiana        Pacers          17   
4084  22200003  2022-10-19 19:00:00        Detroit       Pistons          28   
4083  22200009  2022-10-19 19:30:00        Memphis     Grizzlies          26   
...        ...                  ...            ...           ...         ...   
1477  22301197  2024-04-14 15:30:00    San Antonio         Spurs          22   
1476  22301196  2024-04-14 15:30:00  Oklahoma City       Thunder          23   
1475  22301195  2024-04-14 15:30:00    New Orleans      Pelicans           3   
1474  22301194  2024-04-14 15:30:00      Minnesota  Timberwolves          13   
1473  22301193  2024-04-14 15:30:00        Memphis     Grizzlies          26   

      awayteamCity awayteamName  awayte

In [7]:
id1

array([ 2, 20,  7, ..., 21, 20,  9])

In [8]:
id2

array([15, 25, 22, ...,  8, 25, 10])

In [9]:
outcomes

array([ True,  True, False, ...,  True,  True, False])

In [23]:
start_time = time.time()

num_players = 30
init_rating = np.full(num_players, 1500, dtype=np.float32)
init_rd = np.full(num_players, 350, dtype=np.float32)
init_vol = np.full(num_players, 0.01, dtype=np.float32)

team_wins = np.zeros(num_players)
team_losses = np.zeros(num_players)
team_wr = np.zeros(num_players)

# elo, instant glicko1,2, period glicko1,2
elo = Elo(num_players, init_rating, rating_const=32)
glicko = Glicko(num_players, init_rating, None, init_rd)
glicko2 = Glicko2(num_players, init_rating, None, init_rd, init_vol, 1.2)
glicko_instant = Glicko(num_players, init_rating, None, init_rd)
glicko2_instant = Glicko2(num_players, init_rating, None, init_rd, init_vol, 1.2)

elo_acc = glicko_acc = glicko2_acc = glicko_instant_acc = glicko2_instant_acc = control_acc_wr = control_acc_win = control_acc_home = 0
num_periods = 13
id1_period, id2_period, outcomes_period = np.array_split(id1, num_periods), np.array_split(id2, num_periods), np.array_split(outcomes, num_periods)
j=0
length = 0
count=0

for i in range(len(id1)):
    if i % 1230 == 0:
        team_wins = np.zeros(num_players)
        team_losses = np.zeros(num_players)
        team_wr = np.zeros(num_players)
    if 1230 <= i < 10000:
        if (elo.ratings[id1[i]] >= elo.ratings[id2[i]] and outcomes[i]) or (elo.ratings[id1[i]] < elo.ratings[id2[i]] and not outcomes[i]):
            elo_acc += 1
        if (glicko.ratings[id1[i]] >= glicko.ratings[id2[i]] and outcomes[i]) or (glicko.ratings[id1[i]] < glicko.ratings[id2[i]] and not outcomes[i]):
            glicko_acc += 1
        if (glicko2.ratings[id1[i]] >= glicko2.ratings[id2[i]] and outcomes[i]) or (glicko2.ratings[id1[i]] < glicko2.ratings[id2[i]] and not outcomes[i]):
            glicko2_acc += 1
        if (team_wr[id1[i]] >= team_wr[id2[i]] and outcomes[i]) or (team_wr[id1[i]] < team_wr[id2[i]] and not outcomes[i]):
            control_acc_wr += 1
        if (team_wins[id1[i]] >= team_wins[id2[i]] and outcomes[i]) or (team_wins[id1[i]] < team_wins[id2[i]] and not outcomes[i]):
            control_acc_win += 1
        if (glicko_instant.ratings[id1[i]] >= glicko_instant.ratings[id2[i]] and outcomes[i]) or (glicko_instant.ratings[id1[i]] < glicko_instant.ratings[id2[i]] and not outcomes[i]):
            glicko_instant_acc += 1
        if (glicko2_instant.ratings[id1[i]] >= glicko2_instant.ratings[id2[i]] and outcomes[i]) or (glicko2_instant.ratings[id1[i]] < glicko2_instant.ratings[id2[i]] and not outcomes[i]):
            glicko2_instant_acc += 1
        if outcomes[i]:
            control_acc_home += 1
        count+=1
    
    elo.update(id1[i], id2[i], outcomes[i])
    if i+1 == length + len(id1_period[j]):
        glicko2.update_period(id1_period[j], id2_period[j], outcomes_period[j])
        glicko.update_period(id1_period[j], id2_period[j], outcomes_period[j])
        length += len(id1_period[j])
        j+=1
    glicko_instant.update_instant(id1[i], id2[i], outcomes[i])
    glicko2_instant.update_instant(id1[i], id2[i], outcomes[i])
    if outcomes[i]:
        team_wins[id1[i]] += 1
        team_losses[id2[i]] += 1
    else:
        team_wins[id2[i]] += 1
        team_losses[id1[i]] += 1
    team_wr[id1[i]] = team_wins[id1[i]] / team_losses[id1[i]] if team_losses[id1[i]] != 0 else 1
    team_wr[id2[i]] = team_wins[id2[i]] / team_losses[id2[i]] if team_losses[id2[i]] != 0 else 1
        
# for i in range(len(id1_period)):
#     glicko2.update_period(id1_period[i], id2_period[i], outcomes_period[i])
#     glicko.update_period(id1_period[i], id2_period[i], outcomes_period[i])
   
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.25101780891418457 seconds ---


In [24]:
elo_acc = elo_acc / count * 100
glicko_acc = glicko_acc / count * 100
glicko2_acc = glicko2_acc / count * 100
glicko_instant_acc = glicko_instant_acc / count * 100
glicko2_instant_acc = glicko2_instant_acc / count * 100
control_acc_wr = control_acc_wr / count * 100
control_acc_win = control_acc_win / count * 100
control_acc_home = control_acc_home / count * 100

In [25]:
# print("Elo")
# elo.print_nba_data(names)
# print("Glicko (Period)")
# glicko.print_nba_data(names)
# print("Glicko2 (Period)")
# glicko2.print_nba_data(names)
# print("Glicko (Instant)")
# glicko_instant.print_nba_data(names)
# print("Glicko2 (Instant)")
# glicko2_instant.print_nba_data(names)

print(f"Elo Accuracy: {elo_acc}")
print(f"Glicko (Period) Accuracy: {glicko_acc}")
print(f"Glicko2 (Period) Accuracy: {glicko2_acc}")
print(f"Glicko (Instant) Accuracy: {glicko_instant_acc}")
print(f"Glicko2 (Instant) Accuracy: {glicko2_instant_acc}")
print(f"Control Win Rate Accuracy: {control_acc_wr}")
print(f"Control Wins Accuracy: {control_acc_win}")
print(f"Control Home Team Accuracy: {control_acc_home}")
print(len(id1))

Elo Accuracy: 63.00813008130082
Glicko (Period) Accuracy: 64.8780487804878
Glicko2 (Period) Accuracy: 64.95934959349593
Glicko (Instant) Accuracy: 64.95934959349593
Glicko2 (Instant) Accuracy: 65.1219512195122
Control Win Rate Accuracy: 63.25203252032521
Control Wins Accuracy: 63.65853658536585
Control Home Team Accuracy: 54.3089430894309
2460


In [27]:
def evaluate_glicko2(num_periods, init_vol, sys_vol):
    """Evaluate Glicko2 performance with given parameters"""
    init_rating = np.full(num_players, 1500, dtype=np.float32)
    init_rd = np.full(num_players, 350, dtype=np.float32)
    init_vol = np.full(num_players, init_vol, dtype=np.float32)
    
    glicko2 = Glicko2(num_players, init_rating, None, init_rd, init_vol, sys_vol)
    
    # Split data into periods
    id1_period, id2_period, outcomes_period = np.array_split(id1, num_periods), np.array_split(id2, num_periods), np.array_split(outcomes, num_periods)
    
    accuracy = 0
    count = 0
    j = 0
    length = 0
    
    for i in range(len(id1)):
        if 1230 < i < 10000:
            if (glicko2.ratings[id1[i]] >= glicko2.ratings[id2[i]] and outcomes[i]) or (glicko2.ratings[id1[i]] < glicko2.ratings[id2[i]] and not outcomes[i]):
                accuracy += 1
            count += 1
            
        if i+1 == length + len(id1_period[j]):
            glicko2.update_period(id1_period[j], id2_period[j], outcomes_period[j])
            length += len(id1_period[j])
            j += 1
            
    return (accuracy / count * 100) if count > 0 else 0

# Define parameter ranges
period_range = np.arange(10,30)  # Number of rating periods
init_vol_range = np.arange(0.01, 0.31, 0.01)  # Initial player volatility
sys_vol_range = np.arange(0.1, 1.5, 0.1)  # System volatility constant

# Store results
results = []

# Run grid search
for periods in period_range:
    for init_vol in init_vol_range:
        for sys_vol in sys_vol_range:
            accuracy = evaluate_glicko2(periods, init_vol, sys_vol)
            results.append({
                'num_periods': periods,
                'init_vol': init_vol,
                'sys_vol': sys_vol,
                'accuracy': accuracy
            })
            print(f"Periods: {periods}, Init Vol: {init_vol}, Sys Vol: {sys_vol}, Accuracy: {accuracy:.2f}%")

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

# Find best parameters
best_result = results_df.loc[results_df['accuracy'].idxmax()]
print("\nBest Parameters:")
print(f"Number of Periods: {best_result['num_periods']}")
print(f"Initial Volatility: {best_result['init_vol']}")
print(f"System Volatility: {best_result['sys_vol']}")
print(f"Accuracy: {best_result['accuracy']:.2f}%")


Periods: 10, Init Vol: 0.01, Sys Vol: 0.1, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.2, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.30000000000000004, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.4, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.5, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.6, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.7000000000000001, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.8, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 0.9, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 1.0, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 1.1, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 1.2000000000000002, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 1.3000000000000003, Accuracy: 64.12%
Periods: 10, Init Vol: 0.01, Sys Vol: 1.4000000000000001, Accuracy: 64.12%
Periods: 10, Init Vol: 0.02, Sys Vol: 0.1, Accuracy: 64.12%
Periods: 10, Init Vol: 0

/Users/edwin/Desktop/rankSystem/glicko2.py:62: RuntimeWarning: overflow encountered in exp
  E = 1 / (1 + np.exp(-g * (r - rj)))
/Users/edwin/Desktop/rankSystem/glicko2.py:63: RuntimeWarning: divide by zero encountered in divide
  v = 1 / np.sum(np.square(g) * E * (1 - E))
/Users/edwin/Desktop/rankSystem/glicko2.py:70: RuntimeWarning: invalid value encountered in scalar subtract
  return (np.exp(x) * ((delta * delta) - (rd * rd) - v - np.exp(x)) / (2 * temp * temp)) - (x - a) / (tau * tau)


Periods: 21, Init Vol: 0.23, Sys Vol: 0.5, Accuracy: 64.69%
Periods: 21, Init Vol: 0.23, Sys Vol: 0.6, Accuracy: 64.61%
Periods: 21, Init Vol: 0.23, Sys Vol: 0.7000000000000001, Accuracy: 64.61%
Periods: 21, Init Vol: 0.23, Sys Vol: 0.8, Accuracy: 64.61%
Periods: 21, Init Vol: 0.23, Sys Vol: 0.9, Accuracy: 64.52%
Periods: 21, Init Vol: 0.23, Sys Vol: 1.0, Accuracy: 64.93%
Periods: 21, Init Vol: 0.23, Sys Vol: 1.1, Accuracy: 63.95%
Periods: 21, Init Vol: 0.23, Sys Vol: 1.2000000000000002, Accuracy: 63.55%
Periods: 21, Init Vol: 0.23, Sys Vol: 1.3000000000000003, Accuracy: 63.55%
Periods: 21, Init Vol: 0.23, Sys Vol: 1.4000000000000001, Accuracy: 63.30%
Periods: 21, Init Vol: 0.24000000000000002, Sys Vol: 0.1, Accuracy: 64.61%
Periods: 21, Init Vol: 0.24000000000000002, Sys Vol: 0.2, Accuracy: 64.61%
Periods: 21, Init Vol: 0.24000000000000002, Sys Vol: 0.30000000000000004, Accuracy: 64.61%
Periods: 21, Init Vol: 0.24000000000000002, Sys Vol: 0.4, Accuracy: 64.52%
Periods: 21, Init Vol: 0

/Users/edwin/Desktop/rankSystem/glicko2.py:75: RuntimeWarning: overflow encountered in scalar multiply
  a = np.log(IV * IV)
/Users/edwin/Desktop/rankSystem/glicko2.py:91: RuntimeWarning: invalid value encountered in scalar subtract
  while (np.abs(B - A) > eps):
/Users/edwin/Desktop/rankSystem/glicko2.py:111: RuntimeWarning: divide by zero encountered in scalar divide
  new_rd = 1 / np.sqrt(1 / ((rd * rd) + (new_IV * new_IV)) + 1 / v)
/Users/edwin/Desktop/rankSystem/glicko2.py:62: RuntimeWarning: invalid value encountered in multiply
  E = 1 / (1 + np.exp(-g * (r - rj)))
/Users/edwin/Desktop/rankSystem/glicko2.py:112: RuntimeWarning: invalid value encountered in scalar add
  new_rating = rating + (new_rd * new_rd) * np.sum(g * (outcomes - E))


Periods: 22, Init Vol: 0.28, Sys Vol: 0.2, Accuracy: 63.38%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.30000000000000004, Accuracy: 63.38%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.4, Accuracy: 63.47%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.5, Accuracy: 63.47%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.6, Accuracy: 63.38%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.7000000000000001, Accuracy: 63.38%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.8, Accuracy: 63.55%
Periods: 22, Init Vol: 0.28, Sys Vol: 0.9, Accuracy: 63.47%
Periods: 22, Init Vol: 0.28, Sys Vol: 1.0, Accuracy: 63.47%
Periods: 22, Init Vol: 0.28, Sys Vol: 1.1, Accuracy: 64.04%
Periods: 22, Init Vol: 0.28, Sys Vol: 1.2000000000000002, Accuracy: 63.87%
Periods: 22, Init Vol: 0.28, Sys Vol: 1.3000000000000003, Accuracy: 63.38%
Periods: 22, Init Vol: 0.28, Sys Vol: 1.4000000000000001, Accuracy: 63.47%
Periods: 22, Init Vol: 0.29000000000000004, Sys Vol: 0.1, Accuracy: 63.38%
Periods: 22, Init Vol: 0.29000000000000004, Sys Vol: 0.2, Accuracy: 6

...

Periods: 26, Init Vol: 0.21000000000000002, Sys Vol: 1.1, Accuracy: 64.93%
Periods: 26, Init Vol: 0.21000000000000002, Sys Vol: 1.2000000000000002, Accuracy: 65.09%
Periods: 26, Init Vol: 0.21000000000000002, Sys Vol: 1.3000000000000003, Accuracy: 65.17%
Periods: 26, Init Vol: 0.21000000000000002, Sys Vol: 1.4000000000000001, Accuracy: 63.95%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.1, Accuracy: 64.85%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.2, Accuracy: 64.85%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.30000000000000004, Accuracy: 64.85%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.4, Accuracy: 64.85%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.5, Accuracy: 64.77%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.6, Accuracy: 64.77%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.7000000000000001, Accuracy: 64.77%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.8, Accuracy: 64.93%
Periods: 26, Init Vol: 0.22, Sys Vol: 0.9, Accuracy: 65.01%
Periods: 26, Init Vol: 0.22, Sys Vol: 1.0, Accuracy: 65.17%
Periods: 26, Init Vol: 0

/Users/edwin/Desktop/rankSystem/glicko2.py:163: RuntimeWarning: overflow encountered in cast
  self.ratings[i], self.rating_devs[i], self.volatilities[i] = self.update(


Periods: 27, Init Vol: 0.01, Sys Vol: 1.4000000000000001, Accuracy: 64.77%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.1, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.2, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.30000000000000004, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.4, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.5, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.6, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.7000000000000001, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.8, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 0.9, Accuracy: 64.69%
Periods: 27, Init Vol: 0.02, Sys Vol: 1.0, Accuracy: 64.77%
Periods: 27, Init Vol: 0.02, Sys Vol: 1.1, Accuracy: 64.77%
Periods: 27, Init Vol: 0.02, Sys Vol: 1.2000000000000002, Accuracy: 64.77%
Periods: 27, Init Vol: 0.02, Sys Vol: 1.3000000000000003, Accuracy: 64.77%
Periods: 27, Init Vol: 0.02, Sys Vol: 1.4000000000000001, Accuracy: 64.77%
Periods: 

/Users/edwin/Desktop/rankSystem/glicko2.py:61: RuntimeWarning: overflow encountered in multiply
  g = 1 / np.sqrt(1 + 3 * np.square(rdj) / pi2)


Periods: 28, Init Vol: 0.26, Sys Vol: 0.2, Accuracy: 63.87%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.30000000000000004, Accuracy: 63.87%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.4, Accuracy: 63.95%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.5, Accuracy: 64.04%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.6, Accuracy: 63.95%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.7000000000000001, Accuracy: 64.04%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.8, Accuracy: 64.04%
Periods: 28, Init Vol: 0.26, Sys Vol: 0.9, Accuracy: 64.04%
Periods: 28, Init Vol: 0.26, Sys Vol: 1.0, Accuracy: 63.95%
Periods: 28, Init Vol: 0.26, Sys Vol: 1.1, Accuracy: 62.41%
Periods: 28, Init Vol: 0.26, Sys Vol: 1.2000000000000002, Accuracy: 62.16%
Periods: 28, Init Vol: 0.26, Sys Vol: 1.3000000000000003, Accuracy: 62.25%
Periods: 28, Init Vol: 0.26, Sys Vol: 1.4000000000000001, Accuracy: 0.00%
Periods: 28, Init Vol: 0.27, Sys Vol: 0.1, Accuracy: 64.04%
Periods: 28, Init Vol: 0.27, Sys Vol: 0.2, Accuracy: 64.12%
Periods: 28, Init Vol: 0.

In [ ]:
# Best Parameters:
# Number of Periods: 15.0
# Initial Volatility: 0.18000000000000002
# System Volatility: 1.0
# Accuracy: 64.44%
# 0.5% better